In [1]:
import pandas as pd

In [2]:
dataset=pd.read_csv("insurance_pre.csv")
dataset

,age,sex,bmi,children,smoker,charges
0,19,female,27.900,0,yes,16884.92400
1,18,male,33.770,1,no,1725.55230
2,28,male,33.000,3,no,4449.46200
3,33,male,22.705,0,no,21984.47061
4,32,male,28.880,0,no,3866.85520
...,...,...,...,...,...,...
1333,50,male,30.970,3,no,10600.54830
1334,18,female,31.920,0,no,2205.98080
1335,18,female,36.850,0,no,1629.83350
1336,21,female,25.800,0,no,2007.94500


In [3]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)
dataset

,age,bmi,children,charges,sex_male,smoker_yes
0,19,27.900,0,16884.92400,0,1
1,18,33.770,1,1725.55230,1,0
2,28,33.000,3,4449.46200,1,0
3,33,22.705,0,21984.47061,1,0
4,32,28.880,0,3866.85520,1,0
...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,1,0
1334,18,31.920,0,2205.98080,0,0
1335,18,36.850,0,1629.83350,0,0
1336,21,25.800,0,2007.94500,0,0


In [4]:
dataset.columns

Index(['age', 'bmi', 'children', 'charges', 'sex_male', 'smoker_yes'], dtype='object')

In [5]:
independent=dataset[['age', 'bmi', 'children', 'sex_male', 'smoker_yes']]
independent

,age,bmi,children,sex_male,smoker_yes
0,19,27.900,0,0,1
1,18,33.770,1,1,0
2,28,33.000,3,1,0
3,33,22.705,0,1,0
4,32,28.880,0,1,0
...,...,...,...,...,...
1333,50,30.970,3,1,0
1334,18,31.920,0,0,0
1335,18,36.850,0,0,0
1336,21,25.800,0,0,0


In [6]:
dependent=dataset[['charges']]
dependent

,charges
0,16884.92400
1,1725.55230
2,4449.46200
3,21984.47061
4,3866.85520
...,...
1333,10600.54830
1334,2205.98080
1335,1629.83350
1336,2007.94500


In [7]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
train_independent=sc.fit_transform(independent)

In [8]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

param_grid={'criterion':['squared_error','friedman_mse','poisson'],
           'max_features':[None,'sqrt','log2'],
           'n_estimators':[50,100]}
grid=GridSearchCV(RandomForestRegressor(),param_grid,refit=True,verbose=3,n_jobs=-1)
grid.fit(train_independent,dependent)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


C:\Anaconda3\Lib\site-packages\sklearn\model_selection\_search.py:909: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


GridSearchCV(estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'criterion': ['squared_error', 'friedman_mse',
                                       'poisson'],
                         'max_features': [None, 'sqrt', 'log2'],
                         'n_estimators': [50, 100]},
             verbose=3)

In [9]:
re=grid.cv_results_
print(re)

{'mean_fit_time': array([0.45062141, 1.08914986, 0.41999273, 0.85152841, 0.41478748,
       0.81541429, 0.54070458, 1.16863956, 0.41490226, 0.72108264,
       0.41673689, 0.83324752, 0.61521845, 1.28632107, 0.4746099 ,
       0.94272747, 0.49051619, 0.94230566]), 'std_fit_time': array([0.01755127, 0.04652464, 0.02741756, 0.02920094, 0.00616391,
       0.02843387, 0.02177113, 0.05256692, 0.02059966, 0.05462598,
       0.01168577, 0.02245442, 0.01834373, 0.08224634, 0.04404521,
       0.03641493, 0.01576087, 0.04604785]), 'mean_score_time': array([0.01936669, 0.03125176, 0.01562676, 0.02745519, 0.02187772,
       0.03130751, 0.01846728, 0.02812762, 0.01562686, 0.02812791,
       0.02267518, 0.03125219, 0.01562896, 0.03437824, 0.01562657,
       0.03125238, 0.01562667, 0.0334372 ]), 'std_score_time': array([1.23936331e-02, 2.72924195e-06, 7.77697870e-07, 6.05668076e-03,
       7.65652428e-03, 1.07624312e-04, 5.67801002e-03, 6.24952323e-03,
       1.92160050e-06, 6.25073925e-03, 1.21991951

In [10]:
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.450621,0.017551,0.019367,1.239363e-02,squared_error,None,50,"{'criterion': 'squared_error', 'max_features':...",0.851665,0.772324,0.851596,0.813782,0.837076,0.825289,0.029881,17
1,1.089150,0.046525,0.031252,2.729242e-06,squared_error,None,100,"{'criterion': 'squared_error', 'max_features':...",0.848320,0.773211,0.853734,0.820789,0.838816,0.826974,0.029123,15
2,0.419993,0.027418,0.015627,7.776979e-07,squared_error,sqrt,50,"{'criterion': 'squared_error', 'max_features':...",0.861555,0.777038,0.859991,0.831547,0.841697,0.834366,0.030803,6
3,0.851528,0.029201,0.027455,6.056681e-03,squared_error,sqrt,100,"{'criterion': 'squared_error', 'max_features':...",0.854966,0.783378,0.862367,0.829799,0.841347,0.834372,0.027849,5
4,0.414787,0.006164,0.021878,7.656524e-03,squared_error,log2,50,"{'criterion': 'squared_error', 'max_features':...",0.855917,0.770745,0.860395,0.825825,0.836467,0.829870,0.032143,12
5,0.815414,0.028434,0.031308,1.076243e-04,squared_error,log2,100,"{'criterion': 'squared_error', 'max_features':...",0.857890,0.781648,0.857623,0.833436,0.843493,0.834818,0.028131,2
6,0.540705,0.021771,0.018467,5.678010e-03,friedman_mse,None,50,"{'criterion': 'friedman_mse', 'max_features': ...",0.842581,0.768266,0.855875,0.819529,0.835984,0.824447,0.030431,18
7,1.168640,0.052567,0.028128,6.249523e-03,friedman_mse,None,100,"{'criterion': 'friedman_mse', 'max_features': ...",0.847311,0.772994,0.853019,0.818377,0.838860,0.826112,0.029042,16
8,0.414902,0.020600,0.015627,1.921601e-06,friedman_mse,sqrt,50,"{'criterion': 'friedman_mse', 'max_features': ...",0.859799,0.771404,0.854092,0.825459,0.838716,0.829894,0.031618,11
9,0.721083,0.054626,0.028128,6.250739e-03,friedman_mse,sqrt,100,"{'criterion': 'friedman_mse', 'max_features': ...",0.856993,0.781371,0.858959,0.829085,0.842090,0.833699,0.028322,7


In [11]:
grid.best_params_

{'criterion': 'poisson', 'max_features': 'log2', 'n_estimators': 50}

In [12]:
Y_predict=grid.predict(train_independent)
Y_predict

array([16994.165028 ,  2000.3323678,  4723.240985 , ...,  2543.612953 ,
        2140.204179 , 28359.860025 ])

In [13]:
from sklearn.metrics import r2_score
r2=r2_score(dependent,Y_predict)
r2

0.9742835966987047

In [14]:
preinput=sc.transform([[40,28.5,2,1,0]])
result=grid.predict(preinput)
result

C:\Anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


array([6739.742886])